In [118]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import string
import nltk
import re

df=pd.read_csv(r'train.csv')

In [119]:
sentences=[]
for f in df['tweet']:
    sentences.append(f)

labels=[]
for f in df['label']:
    labels.append(f)


          id                                              tweet
0      31963  #studiolife #aislife #requires #passion #dedic...
1      31964   @user #white #supremacists want everyone to s...
2      31965  safe ways to heal your #acne!!    #altwaystohe...
3      31966  is the hp and the cursed child book up for res...
4      31967    3rd #bihday to my amazing, hilarious #nephew...
...      ...                                                ...
17192  49155  thought factory: left-right polarisation! #tru...
17193  49156  feeling like a mermaid ð #hairflip #neverre...
17194  49157  #hillary #campaigned today in #ohio((omg)) &am...
17195  49158  happy, at work conference: right mindset leads...
17196  49159  my   song "so glad" free download!  #shoegaze ...

[17197 rows x 2 columns]


In [120]:
from nltk.corpus import stopwords
stopword=set(stopwords.words('english'))
print(stopword)
def lowercase(text):
    return text.lower()
def remove_punctuation(text):
    return text.translate(str.maketrans('', '',string.punctuation))
def remove_stopwords(text):
    new_word=[]
    for word in str(text).split():
        if word not in stopword:
            new_word.append(word)
    return " ".join(new_word)
def remove_url(text):
    url_pattern=re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'',text)
def remove_extra(text):
    new_word=[]
    for word in str(text).split():
        if word.isalpha():
                    new_word.append(word)
    return  " ".join(new_word)        

for i in range (len(sentences)):
    
    sentences[i]=lowercase(sentences[i])
    sentences[i]=remove_url(sentences[i])
    sentences[i]=remove_stopwords(sentences[i])
    sentences[i]=remove_punctuation(sentences[i])
    sentences[i]=remove_extra(sentences[i])
    



{'few', 'for', 'hers', 'was', 'isn', 'it', 'am', 'further', 'own', "won't", 'wouldn', 'but', 'some', 'does', 's', "you're", 'is', 'they', 'before', 'this', 'd', 'aren', "weren't", 'your', "you'd", 'ourselves', 'that', 'can', 'itself', 'myself', "don't", 'having', 'against', 'of', 'when', 'as', 're', "she's", 'where', 'doing', 'them', 'and', 'there', 'ma', 'm', 'me', 'now', 've', 'until', 'don', 'very', 'nor', "hadn't", 'by', "needn't", 'here', 'our', 'ain', 'only', "hasn't", 'weren', 'below', 'while', "wouldn't", 'were', 'with', 'won', "shan't", 'the', 'about', 'o', 'above', "should've", 'will', 'shan', 'yourself', 'hasn', 'him', "doesn't", 'ours', 'any', 'didn', 'who', 'off', 'after', 'why', 'shouldn', 'under', 'from', 'at', 'have', 'yourselves', 'more', 'hadn', "mightn't", 'do', 'needn', 'which', 'theirs', 'an', "isn't", "wasn't", 'doesn', "you've", 'did', 'on', 't', 'up', 'other', 'through', "shouldn't", 'you', 'he', 'each', "mustn't", 'if', 'between', 'during', 'in', 'y', 'no', 'sh

In [100]:
print(sentences[13])

user cnn calls michigan middle school build wall chant tcot


In [121]:
input_size=31962
seq_len=70
pad='post'
trunc='post'

In [122]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


training_sentences= sentences[0:25000]
training_labels=labels[0:25000]

test_sentences=sentences[25000:]
test_labels=labels[25000:]


tokenizer=Tokenizer(num_words=5000,oov_token='<OOV>')
tokenizer.fit_on_texts(training_sentences)

word_index=tokenizer.word_index
#print(word_index)

training_sequences=tokenizer.texts_to_sequences(training_sentences)
training_padded = pad_sequences(training_sequences, maxlen=seq_len, padding=pad, truncating=trunc)
#print(training_padded[1])
testing_sequences = tokenizer.texts_to_sequences(test_sentences)
testing_padded = pad_sequences(testing_sequences, maxlen=seq_len, padding=pad, truncating=pad)

In [123]:
training_padded = np.array(training_padded)
training_labels = np.array(training_labels)
testing_padded = np.array(testing_padded)
test_labels = np.array(test_labels)

In [124]:
model=tf.keras.Sequential([
    tf.keras.layers.Embedding(input_size,8,input_length=seq_len),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24,activation='relu'),
    tf.keras.layers.Dense(1,activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

history = model.fit(training_padded, training_labels, epochs=25, validation_data=(testing_padded, test_labels), verbose=2)

Epoch 1/30
782/782 - 1s - loss: 0.2923 - accuracy: 0.9291 - val_loss: 0.2457 - val_accuracy: 0.9299
Epoch 2/30
782/782 - 1s - loss: 0.2338 - accuracy: 0.9298 - val_loss: 0.2218 - val_accuracy: 0.9299
Epoch 3/30
782/782 - 1s - loss: 0.1892 - accuracy: 0.9318 - val_loss: 0.1635 - val_accuracy: 0.9379
Epoch 4/30
782/782 - 1s - loss: 0.1272 - accuracy: 0.9506 - val_loss: 0.1364 - val_accuracy: 0.9479
Epoch 5/30
782/782 - 1s - loss: 0.1008 - accuracy: 0.9624 - val_loss: 0.1321 - val_accuracy: 0.9537
Epoch 6/30
782/782 - 1s - loss: 0.0895 - accuracy: 0.9666 - val_loss: 0.1309 - val_accuracy: 0.9540
Epoch 7/30
782/782 - 1s - loss: 0.0821 - accuracy: 0.9703 - val_loss: 0.1320 - val_accuracy: 0.9560
Epoch 8/30
782/782 - 1s - loss: 0.0754 - accuracy: 0.9730 - val_loss: 0.1366 - val_accuracy: 0.9553
Epoch 9/30
782/782 - 1s - loss: 0.0697 - accuracy: 0.9749 - val_loss: 0.1367 - val_accuracy: 0.9550
Epoch 10/30
782/782 - 1s - loss: 0.0653 - accuracy: 0.9775 - val_loss: 0.1417 - val_accuracy: 0.9552

In [129]:
d=pd.read_csv(r'test.csv')
print(d)

test_sentences=[]
for f in d['tweet']:
    test_sentences.append(f)
print(len(test_sentences))
for i in range (len(test_sentences)):
    
    test_sentences[i]=lowercase(test_sentences[i])
    test_sentences[i]=remove_url(test_sentences[i])
    test_sentences[i]=remove_stopwords(test_sentences[i])
    test_sentences[i]=remove_punctuation(test_sentences[i])
    test_sentences[i]=remove_extra(test_sentences[i])    
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, maxlen=seq_len, padding=pad, truncating=pad)

test_padded = np.array(test_padded)

p=model.predict(test_padded)

d['labels']=p
d.to_csv("test.csv")

          id                                              tweet
0      31963  #studiolife #aislife #requires #passion #dedic...
1      31964   @user #white #supremacists want everyone to s...
2      31965  safe ways to heal your #acne!!    #altwaystohe...
3      31966  is the hp and the cursed child book up for res...
4      31967    3rd #bihday to my amazing, hilarious #nephew...
...      ...                                                ...
17192  49155  thought factory: left-right polarisation! #tru...
17193  49156  feeling like a mermaid ð #hairflip #neverre...
17194  49157  #hillary #campaigned today in #ohio((omg)) &am...
17195  49158  happy, at work conference: right mindset leads...
17196  49159  my   song "so glad" free download!  #shoegaze ...

[17197 rows x 2 columns]
17197
